# Расчет цены на автомобиль
### Книга 02 - сбор данных с сайта авто-ру.

Постановка задачи:
Необходимо разработать модель, которая бы рассчитывала цену на выставленный на продажу автомобиль.  По условиям учебной задачи обучающих данных в явном виде не предоставлено. Только тестовые, собранные на авто-ру больше года назад. Необходимо самостоятельно разработать программу, которая бы собирала данные по объявлениям на том же сайте авто.ру. Дополнительная сложность - количество данных. Оцениваться работа будет по порядка 35к записей. Необходимо собрать порядка 140 тыс записей.  На самом сайте автору сейчас актуально порядка 90к объявлений.

Поиск в сети сформировал следующую картину:
- Есть готовая работающая программа сбора данных с сайта auto-ru, требующая косметической доработки. Автор (rzabolotin) выложил ее на github в составе своего решения. Программа сформирована для самостоятельного запуска интерпретатором Python и вызывает вопросы в части сбора данных.
- Существуют данные, собранные rzabolotin примерно 21-01-08, которые содержат дубликаты. После отсева остается примерно 35к записей.
- Существуют данные от авторов учебной задачи, которые сохранены совсем в ином формате и вероятно собраны до 20-09-09.
- На kaggle найден еще один источник данных от Kiril & Tanya, датированный примерно 21-08-24.
- Данные теста действительно частично устарели - записи не находятся.

Ближайший план работы:
- Перенести программу сбора данных в ноутбук из соображений большей интерактивности работы в среде DataSpell-EAP, внести в нее косметические коррективы и вероятно исправить стратегию сбора данных.
- Собрать какое-то количество данных по состоянию на текущий момент.
- Объединить воедино наборы данных от авторов учебной задачи, rzabolotin, kiril&tanyaб и мои собственные.

# 1. Сбор данных с сайта auto.ru


#### Декларации и импорт библиотек

Копирование в ноутбук найденной унаследованной (legacy) программы парсинга сайта.  В ней понадобится сделать исправления - привести в соответствие с новой структурой сайта.

In [1]:
import argparse
from collections import namedtuple, defaultdict
from csv import DictWriter
import json
from pathlib import Path
import time, random
import pandas as pd

import requests
from loguru import logger

In [3]:
def get_headers():
    headers = '''
Host: auto.ru
Connection: keep-alive
Content-Length: 99
x-requested-with: fetch
x-client-date: 1603066469874
x-csrf-token: c23073bb4cd65413662a41bd460fd8317459fe3ce6d83db1
x-page-request-id: 3c4800b60eb9e8c568e5a515f5cd4872
content-type: application/json
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36
x-client-app-version: 202010.16.122434
Accept: */*
Origin: https://auto.ru
Sec-Fetch-Site: same-origin
Sec-Fetch-Mode: same-origin
Sec-Fetch-Dest: empty
Referer: https://auto.ru/cars/bmw/all/?output_type=list&page=1
Accept-Encoding: gzip, deflate, br
Accept-Language: ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
Cookie: autoru_sid=a%3Ag5f88f7d72b1bif6t0m99h0krigjgs2e.a783a85b576c8a7acaea4faafaa81ffc%7C1602811863959.604800.QhhqH0HWfM4BPWrsvjyfIg.8EXrpUR7Bq1a2gOSCnsb0HnXxvmHbYB9eF5Uz5o_bZE; autoruuid=g5f88f7d72b1bif6t0m99h0krigjgs2e.a783a85b576c8a7acaea4faafaa81ffc; suid=63abf8672f4a9e550bb96dd00b95ad21.8e155bfb838a5227bdd9dea1d2cbdc3e; _ym_uid=1602811867719278329; yuidcs=1; crookie=PZwS3/iYq2PFIw/dbrsqDVB/0e2v79Xe/8RsG6ySC8Djcl+mh/UCjYohgODaSkw7rMa6O9v7+RD56YSQKE2fhSkWxV8=; cmtchd=MTYwMjgyODMxMjMwNQ==; bltsr=1; yuidlt=1; yandexuid=360949521578055883; my=YwA%3D; _ym_isad=1; promo-app-banner-shown=1; promo-header-counter=4; _csrf_token=c23073bb4cd65413662a41bd460fd8317459fe3ce6d83db1; from=direct; X-Vertis-DC=myt; _ym_wasSynced=%7B%22time%22%3A1603066366683%2C%22params%22%3A%7B%22eu%22%3A0%7D%2C%22bkParams%22%3A%7B%7D%7D; gdpr=0; _ym_visorc_22753222=b; from_lifetime=1603066463083; _ym_d=1603066470'''
    headers = {line.split(': ')[0]: line.split(': ')[1] for line in headers.strip().split('\n')}
    return headers


In [4]:
def get_data_from_site(pange_num: int, brand: str) -> dict:
    """ получает данные о автомобилях с авто.ру, номер страницы передан в параметре page_num
    возвращает полученный с сайта json """

    base_url = 'https://auto.ru/-/ajax/desktop/listing/'
    params = dict(category='cars', section="all", output_type="list", page=pange_num,
                  catalog_filter=[{'mark': brand}], geo_id=[213], geo_radius=800)
    r = requests.post(base_url, json=params, headers=get_headers())
    r.raise_for_status()
    return r.json()

In [47]:
def add_car_data(car_data: list, data_json: dict):
    """ разбирает данные из переданного объекта data_json
    из массива offers, формирует объекты CarInfo и
    добавляет их в список car_data"""

    if 'offers' not in data_json:
        return

    for car in data_json['offers']:
        if 'configuration' not in car['vehicle_info']:
            continue

        body_type_human = car['vehicle_info']['configuration']['human_name']
        body_type = car['vehicle_info']['configuration']['body_type']
        transmission = car['vehicle_info']['tech_param']['transmission']
        engine_volume = car['vehicle_info']['tech_param']['displacement']
        engine_volume = round(float(engine_volume) / 1000, 1)

        try:
            purchase_date = car['documents']['purchase_date']
        except KeyError:
            purchase_date = None

        try:
            owners_number = car['documents']['owners_number']
        except KeyError:
            owners_number = None

        try:
            description = car['description']
        except KeyError:
            description = None
        try:
            pts = car['documents']['pts']
        except KeyError:
            pts = None
        try:
            price = car['price_info']['RUR']
        except KeyError:
            continue

        brand = car['vehicle_info']['mark_info']['name']
        # model = car['vehicle_info']['model_info']['name']
        model = car['vehicle_info']['model_info']['code']
        sell_id = car['saleId']
        section = car['section']
        # Здесь ошибка - сохраняются адреса в старом формате ссылок авто-ру,
        # в частности "5 серии" вместо текущей "5er"
        car_url = f'https://auto.ru/cars/{section}/sale/{brand.lower()}/{model.lower()}/{sell_id}/'

        info = CarInfo(bodyType=body_type_human,
                       brand=brand,
                       car_url=car_url,
                       image=car['state']['image_urls'][0]['sizes']['small'],
                       color=car['color_hex'],
                       complectation_dict=car['vehicle_info']['complectation'],
                       equipment_dict=car['vehicle_info']['equipment'],
                       model_info=car['vehicle_info']['model_info'],
                       model_name=model,
                       location=car['seller']['location']['region_info']['name'],
                       parsing_unixtime=car['additional_info']['fresh_date'],
                       priceCurrency=car['price_info']['currency'],
                       sell_id=sell_id,
                       super_gen=car['vehicle_info']['super_gen'],
                       vendor=car['vehicle_info']['vendor'],
                       fuelType=car['vehicle_info']['tech_param']['engine_type'],
                       modelDate=car['vehicle_info']['super_gen']['year_from'],
                       name=car['vehicle_info']['tech_param']['human_name'],
                       numberOfDoors=car['vehicle_info']['configuration']['doors_count'],
                       productionDate=car['documents']['year'],
                       vehicleConfiguration=body_type + " " + transmission + " " + str(engine_volume),
                       vehicleTransmission=transmission,
                       engineDisplacement=str(engine_volume) + ' LTR',
                       enginePower=str(car['vehicle_info']['tech_param']['power']) + ' N12',
                       description=description,
                       mileage=car['state']['mileage'],
                       Привод=car['vehicle_info']['tech_param']['gear_type'],
                       Руль=car['vehicle_info']['steering_wheel'],
                       Состояние=car['state']['state_not_beaten'],
                       Владельцы=owners_number,
                       ПТС=pts,
                       Таможня=car['documents']['custom_cleared'],
                       Владение=purchase_date,
                       price=price
                       )
        car_data.append(info)


In [41]:
def write_to_csv(car_data: list, output_folder_path: Path, page_num: int):
    """Записывает информацию о машинах из car_data в файл формата csv
    output_folder_path задает папку для сохранения
    page_num - нужен для наименования файла"""

    if not car_data:
        return

    filename = output_folder_path / f'train_{page_num}.csv'
    filename = output_folder_path / f'data.csv'
    #
    with open(filename, 'a', encoding='utf-8', newline='') as f:
        logger.info(f'writing {filename}, содержащий {len(car_data)} записей')
        writer = DictWriter(f, fieldnames=columns)
        writer.writeheader()
        writer.writerows([car._asdict() for car in car_data])
        f.close()

In [7]:
def pickup_brand(brand_stats: dict) -> str:
    cars_count = sum(val for val in brand_stats.values())
    brand_proportion = {key: val / cars_count for key, val in brand_stats.items()}

    proportion_in_test_copy = proportion_in_test.copy()
    for key, val in brand_proportion.items():
        proportion_in_test_copy[key] -= val

    return sorted(proportion_in_test_copy.items(),
                  key=lambda x: x[1],
                  reverse=True)[0][0] # first element, key of pair key:value


In [8]:
"""
LEGACY  Function!!!  Сделаем ее некомпилируемой - буква d в определении имени функции пропущена.
def parse_data(n_pages, output_folder, save_json, json_folder):
    """Входная точка в программу. Содержит верхнеуровневую логику.
    Собирает информацию с сайта, парсит и сохраняет в формате csv"""

    logger.info('Starting parse data from auto.ru about cars')

    if save_json:
        json_folder_path = Path(json_folder)
        json_folder_path.mkdir(parents=True, exist_ok=True)

    output_folder_path = Path(output_folder)
    output_folder_path.mkdir(parents=True, exist_ok=True)

    car_data = []
    brand_stats = defaultdict(lambda: 1)

    for page_num in range(1, n_pages + 1):

        brand = pickup_brand(brand_stats)

        logger.info(f'processing page: {page_num} {brand} {brand_stats[brand]}')
        try:
            data_json = get_data_from_site(brand_stats[brand], brand)
        except Exception as e:
            logger.error(f"Error in parsing: {e}")
            continue

        brand_stats[brand] += 1

        if save_json:
            with open(json_folder_path / f'page_{page_num}.json', 'w', encoding='utf-8') as f:
                json.dump(data_json, f)

        add_car_data(car_data, data_json)

        if page_num % 50 == 0:
            write_to_csv(car_data, output_folder_path, page_num)
            car_data = []

    write_to_csv(car_data, output_folder_path, page_num)

    logger.info('parsing successfully finished')
"""

SyntaxError: invalid syntax (819779792.py, line 2)

In [9]:
"""
Это вызов кода в оригинальной отдельной программе.
Сохранен для истории
Автор предлагал вызывать программу из интерпретатора командной строки

python autoru_parser.ry -o outputdirectory  NNN


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("n_pages", help="number of pages to parse", type=int)
    parser.add_argument("-o", "--output_folder", help='folder to save parsed csv files', type=str, default='.')
    parser.add_argument("-j", "--save_json", help="save json files", action="store_true")
    parser.add_argument("--json_folder", help='folder for json files', type=str, default='json')
    args = parser.parse_args()

    parse_data(args.n_pages,
               args.output_folder,
               args.save_json,
               args.json_folder)


if __name__ == '__main__':
    main()
"""

'\nЭто вызов кода в оригинальной отдельной программе.\nСохранен для истории\nАвтор предлагал вызывать программу из интерпретатора командной строки\n\npython autoru_parser.ry -o outputdirectory  NNN\n\n\ndef main():\n    parser = argparse.ArgumentParser()\n    parser.add_argument("n_pages", help="number of pages to parse", type=int)\n    parser.add_argument("-o", "--output_folder", help=\'folder to save parsed csv files\', type=str, default=\'.\')\n    parser.add_argument("-j", "--save_json", help="save json files", action="store_true")\n    parser.add_argument("--json_folder", help=\'folder for json files\', type=str, default=\'json\')\n    args = parser.parse_args()\n\n    parse_data(args.n_pages,\n               args.output_folder,\n               args.save_json,\n               args.json_folder)\n\n\nif __name__ == \'__main__\':\n    main()\n'

In [10]:
"""
Это определения из унаследованной программы. Часть из них будет иметь значение
"""
columns = ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement',
           'enginePower', 'location',
           'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name',
           'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen',
           'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод',
           'Руль', 'Состояние', 'Таможня', 'price']

CarInfo = namedtuple('Car', columns)

brand_in_test = {
    'BMW': 4473,
    'VOLKSWAGEN': 4404,
    'NISSAN': 4393,
    'MERCEDES': 4180,
    'TOYOTA': 3913,
    'AUDI': 3421,
    'MITSUBISHI': 2843,
    'SKODA': 2741,
    'VOLVO': 1463,
    'HONDA': 1150,
    'INFINITI': 871,
    'LEXUS': 834
}
cars_in_test = sum(val for val in brand_in_test.values())
proportion_in_test = {key: val / cars_in_test for key, val in brand_in_test.items()}

На этом legacy код закончился.  Для сбора данных я делаю новую процедуру по аналогии с унаследованной, в которой отключаю json и прочие бантики.

In [14]:
def parse_data_02(n_pages, output_folder, brand ): # , save_json, json_folder):

    logger.info('Starting parse data from auto.ru about cars')


    output_folder_path = Path(output_folder)
    output_folder_path.mkdir(parents=True, exist_ok=True)

    car_data = []
    brand_stats = defaultdict(lambda: 1)

    for page_num in range(1, n_pages + 1):


        logger.info(f'processing page: {page_num} {brand}) # {brand_stats[brand]}')
        try:
            data_json = get_data_from_site(page_num, brand)
        except Exception as e:
            logger.error(f"Error in parsing: {e}")
            continue

        add_car_data(car_data, data_json)

        if page_num % 50 == 0:
            write_to_csv(car_data, output_folder_path, page_num)
            car_data = []

        time.sleep(random.random()*2)

    write_to_csv(car_data, output_folder_path, page_num)

    logger.info('parsing successfully finished')


In [24]:
parse_data_02(99,'output','AUDI')

2021-11-20 18:18:47.182 | INFO     | __main__:parse_data_02:5 - Starting parse data from auto.ru about cars
2021-11-20 18:18:47.184 | INFO     | __main__:parse_data_02:21 - processing page: 1 AUDI) # 1
2021-11-20 18:18:48.106 | INFO     | __main__:parse_data_02:21 - processing page: 2 AUDI) # 1
2021-11-20 18:18:48.941 | INFO     | __main__:parse_data_02:21 - processing page: 3 AUDI) # 1
2021-11-20 18:18:49.978 | INFO     | __main__:parse_data_02:21 - processing page: 4 AUDI) # 1
2021-11-20 18:18:50.817 | INFO     | __main__:parse_data_02:21 - processing page: 5 AUDI) # 1
2021-11-20 18:18:51.797 | INFO     | __main__:parse_data_02:21 - processing page: 6 AUDI) # 1
2021-11-20 18:18:52.823 | INFO     | __main__:parse_data_02:21 - processing page: 7 AUDI) # 1
2021-11-20 18:18:53.658 | INFO     | __main__:parse_data_02:21 - processing page: 8 AUDI) # 1
2021-11-20 18:18:54.699 | INFO     | __main__:parse_data_02:21 - processing page: 9 AUDI) # 1
2021-11-20 18:18:55.822 | INFO     | __main__:

Проба прошла отлично.  Стерты еще несколько проб.  По ходу сбора данных возникло несколько вопросов.

1.Как собирать данные - все значимые марки (по мнению автора условий) или только марки, которые есть в тестовом наборе?

2. На сайте автору отображается информация на 99 страницах, но данных по некоторым маркам больше. Как скачать максимальное количество данных по маркам?

Принято два решения:

1. Сделать два набора данных - все значимые марки (36 брендов) и выборочные марки для теста (12 брендов).  Потом в ходе экспериментов понять важность 24 лишних брендов.
2. Скачивать весь объем данных по маркам несколько раз с перерывами 4-8 часов. Потом объединить эти наборы в один и устранить дубликаты.  Результат купажа и сепажа сохранить в виде файла для дальнейшего использования.

In [52]:
brands = ['AUDI', 'BMW', 'CADILLAC', 'CHERY', 'CHEVROLET', 'CHRYSLER', 'CITROEN', 'DAEWOO', 'DODGE', 'FORD', 'GEELY', 'HONDA', 'HYUNDAI', 'INFINITI', 'JAGUAR', 'JEEP', 'KIA', 'LEXUS', 'MAZDA', 'MINI', 'MITSUBISHI', 'NISSAN', 'OPEL', 'PEUGEOT', 'PORSCHE', 'RENAULT', 'SKODA', 'SUBARU', 'SUZUKI', 'TOYOTA', 'VOLKSWAGEN', 'VOLVO', 'GREAT_WALL', 'LAND_ROVER', 'MERCEDES', 'SSANG_YONG']

In [61]:
for b in brand_in_test:   # или brand_in_test
    parse_data_02(99,'out211120',b)
    time.sleep(random.random()*10)

2021-11-21 19:19:15.070 | INFO     | __main__:parse_data_02:5 - Starting parse data from auto.ru about cars
2021-11-21 19:19:15.071 | INFO     | __main__:parse_data_02:21 - processing page: 1 BMW) # 1
2021-11-21 19:19:15.950 | INFO     | __main__:parse_data_02:21 - processing page: 2 BMW) # 1
2021-11-21 19:19:17.115 | INFO     | __main__:parse_data_02:21 - processing page: 3 BMW) # 1
2021-11-21 19:19:18.138 | INFO     | __main__:parse_data_02:21 - processing page: 4 BMW) # 1
2021-11-21 19:19:19.140 | INFO     | __main__:parse_data_02:21 - processing page: 5 BMW) # 1
2021-11-21 19:19:20.395 | INFO     | __main__:parse_data_02:21 - processing page: 6 BMW) # 1
2021-11-21 19:19:21.451 | INFO     | __main__:parse_data_02:21 - processing page: 7 BMW) # 1
2021-11-21 19:19:22.179 | INFO     | __main__:parse_data_02:21 - processing page: 8 BMW) # 1
2021-11-21 19:19:23.018 | INFO     | __main__:parse_data_02:21 - processing page: 9 BMW) # 1
2021-11-21 19:19:24.280 | INFO     | __main__:parse_dat

In [7]:
df = pd.read_csv('out211120/data_1200.csv')
for n in ['data-1201.csv', 'data-1202.csv', 'data-1203.csv']:
    df1 = pd.read_csv('out211120/'+n)
    df = df.append(df1)
    df.drop_duplicates(['sell_id'], inplace=True)
df.reset_index(inplace=True)

In [8]:
df.shape

(42046, 35)

In [9]:
df.head()

,index,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,location,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,0,Внедорожник 5 дв.,BMW,https://auto.ru/cars/used/sale/bmw/x5/11057182...,040001,{'id': '0'},Автомобиль приобретался новым и с тех пор нахо...,3.0 LTR,249 N12,Москва,...,AUTOMATIC,EUROPEAN,2,"{'year': 2017, 'month': 5}",ORIGINAL,ALL_WHEEL_DRIVE,LEFT,True,True,4350000
1,1,Седан,BMW,https://auto.ru/cars/used/sale/bmw/3er/1106015...,040001,"{'id': '9361242', 'name': '316i SE', 'availabl...",Юридически чистый и технически обслуженный авт...,1.6 LTR,136 N12,Щелково,...,AUTOMATIC,EUROPEAN,3,"{'year': 2020, 'month': 5}",ORIGINAL,REAR_DRIVE,LEFT,True,True,1420000
2,2,Седан,BMW,https://auto.ru/cars/used/sale/bmw/m5/11057329...,CACECB,{'id': '0'},Продажа указанного БМВ осуществляется в формат...,4.4 LTR,600 N12,Москва,...,ROBOT,EUROPEAN,1,"{'year': 2014, 'month': 12}",ORIGINAL,REAR_DRIVE,LEFT,True,True,80000000
3,3,Купе,BMW,https://auto.ru/cars/used/sale/bmw/3er/1103184...,EE1D19,{'id': '0'},Бмв 316 по документам 1.8\nПо факту был устано...,1.8 LTR,90 N12,Москва,...,MECHANICAL,EUROPEAN,4,"{'year': 2017, 'month': 7}",DUPLICATE,REAR_DRIVE,LEFT,True,True,990000
4,4,Седан,BMW,https://auto.ru/cars/used/sale/bmw/7er/1105925...,040001,{'id': '0'},"Автомобиль в достойном состоянии. Чистый, не к...",4.8 LTR,367 N12,Москва,...,AUTOMATIC,EUROPEAN,2,"{'year': 2019, 'month': 1}",ORIGINAL,REAR_DRIVE,LEFT,True,True,645000


In [10]:
df.loc[df.bodyType=='bodytype'].head()

,index,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,location,...,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price


In [4]:
df.to_csv('out211120/211121-12brands.csv')

In [5]:
for n in ['data-360.csv', 'data-361.csv', 'data-362.csv']:
    df1 = pd.read_csv('out211120/'+n)
    df = df.append(df1)
    df.drop_duplicates(['sell_id'], inplace=True)
df.reset_index(inplace=True)
df.shape

(91000, 36)

In [6]:
df.to_csv('out211120/211121-36brands.csv')

 ### Предварительные выводы.

Собрано 42к записей по 12 брендам и 91к записей по 36 брендам, что выше результата, предложенного в качестве baseline авторами условий.  Идеи про объединение с унаследованными наборами, пока отложу на стадию возможных улучшений.  Внимание будет сосредоточено на создании собственного baseline MVP.